# ORM

___

## ORM

ORM: Class와 Table의 결합.

DBMS가 아닌 Programming Technique - ORM과 RDBMS는 같이 존재해야 한다.

ORM 사용 이유
1. DB를 Business Model(Logic)과 자동으로 결합(binding)해 준다.
2. 웹 형태를 만들때 유용함.

비즈니스 로직에 따라 만든 것과 현실은 매우 다르다 => 클래스와 테이블을 맵핑하는 ORM 등장.

ORM은 SQL 구성을 대신 해주는 것이지 DBMS가 아니다.

___

## SQLAlchemy

- Engine Part(Core Part): Connection, Dialect - 각 RDBMS에 맞는 connection을 정의(추상화한 것)
- ORM Part: 클래스와 DB 맵핑 - Core에 지시. 추상화된 형태로 쓰는 것은 ORM, 실제 작업은 Core.

Engine
- 작업 시작점.
- DB접속정보 기억.
- SQL문 해석하여 주고받는 역할.
- SQL 생성까지만 담당(실행은 안함).
- 주로 백엔드 관련 작업 담당.

___

### Base

In [3]:
import sqlalchemy

In [200]:
from sqlalchemy import create_engine

In [207]:
engine_review = create_engine('sqlite:///alchemy.db', echo=True)

In [208]:
print(engine_review)

Engine(sqlite:///alchemy.db)


create_engine('sqlite:username:password@host:port/database', echo=True)

echo: 명령 실행후 반응을 보려면 True, 아니면 False

In [209]:
from sqlalchemy import MetaData

In [210]:
metadata = MetaData()

MetaData(): 현재 엔진의 정보를 불러옴.

In [211]:
print(metadata)

MetaData(bind=None)


현재 엔진에 아무 접속도 되어있지 않은 상태

In [212]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

metadata = MetaData()
users = Table('users', metadata,
    Column('id', Integer, primary_key = True),
    Column('name', String),
    Column('fullname', String)
)

addresses = Table('addresses', metadata,
    Column('id', Integer, primary_key = True),
    Column('user_id', Integer, ForeignKey('users.id')),
    Column('email_address', String, nullable = False)
)

metadata.create_all(engine_review)

2018-07-15 20:59:19,736 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-15 20:59:19,738 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 20:59:19,741 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-15 20:59:19,743 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 20:59:19,745 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-07-15 20:59:19,747 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 20:59:19,752 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2018-07-15 20:59:19,757 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 20:59:19,759 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2018-07-15 20:59:19,761 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 20:59:19,766 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-15 20:59:19,769 INFO sqlalchemy.engine.b

users, addresses라는 테이블 생성

metadata.create_all(engine_review): 만든 테이블을 engine_review에 바인딩.

=> 입력한 내용을 SQL문으로 바꿔주고 commit까지 끝낸다.

In [213]:
for row in metadata.tables:
    print(row)

users
addresses


In [214]:
print(metadata)

MetaData(bind=None)


ORM의 장점: SQL을 정확히 몰라도 사용 가능하다 => 작업하기 수월해짐.

___

### Insert

In [128]:
users.insert()

In [129]:
print(users.insert())

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)


insert 리턴값: SQL문이 아닌 Object.

In [130]:
insert = users.insert().values(name='kim', fullname='Anonymous, Kim')

In [216]:
print(users.insert().values(name='kim', fullname='Anonymous, Kim'))

INSERT INTO users (name, fullname) VALUES (:name, :fullname)


인자를 넣어주면 자동으로 SQL문으로 바꿔준다.

In [217]:
insert.compile().params

{'id': None, 'name': None, 'fullname': None}

인자로 준 정보가 어떻게 바뀌어 들어가는지 볼수 있음

In [218]:
insert.bind = engine_review

conn = engine_review.connect()
result = conn.execute(insert)

2018-07-15 21:05:02,328 INFO sqlalchemy.engine.base.Engine INSERT INTO users DEFAULT VALUES
2018-07-15 21:05:02,330 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:05:02,333 INFO sqlalchemy.engine.base.Engine COMMIT


engine_review에 바인딩 된 insert 문을 engine_review에 연결하고 실행한다.

In [219]:
result.inserted_primary_key

[1]

.inserted_primary_key: 마지막으로 실행된 결과 중 primary key 값을 가져온다.

In [220]:
insert = users.insert()

result = conn.execute(insert, name='lee', fullname='Unknown, Lee')
result = conn.execute(insert, name='xx', fullname='xxxx, xxx')

2018-07-15 21:06:04,265 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2018-07-15 21:06:04,266 INFO sqlalchemy.engine.base.Engine ('lee', 'Unknown, Lee')
2018-07-15 21:06:04,268 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-15 21:06:04,272 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2018-07-15 21:06:04,274 INFO sqlalchemy.engine.base.Engine ('xx', 'xxxx, xxx')
2018-07-15 21:06:04,276 INFO sqlalchemy.engine.base.Engine COMMIT


In [221]:
result.inserted_primary_key

[3]

In [222]:
conn.execute(addresses.insert(), [
    {'user_id':1, 'email_address':'anonymous.kim@test,com'},
    {'user_id':2, 'email_address':'unknown.lee@test.com'}
])

2018-07-15 21:06:53,399 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2018-07-15 21:06:53,401 INFO sqlalchemy.engine.base.Engine ((1, 'anonymous.kim@test,com'), (2, 'unknown.lee@test.com'))
2018-07-15 21:06:53,404 INFO sqlalchemy.engine.base.Engine COMMIT


insert를 실행할 테이블을 지정하여 실행하는것도 가능.

___

### Select

In [223]:
from sqlalchemy.sql import select

query = select([users])
print(query)

SELECT users.id, users.name, users.fullname 
FROM users


테이블의 컬럼 명을 따로 명시할 경우 []빼도 되지만 아닐 경우 []를 꼭 써줘야 실행된다.

In [224]:
result = conn.execute(query)

2018-07-15 21:08:04,334 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-07-15 21:08:04,337 INFO sqlalchemy.engine.base.Engine ()


In [225]:
for row in result:
    print(row)

(1, None, None)
(2, 'lee', 'Unknown, Lee')
(3, 'xx', 'xxxx, xxx')


반복문을 활용하여 테이블을 한줄씩 출력.

In [226]:
result = conn.execute(query)
row = result.fetchone()
print(row)
row = result.fetchone()
print(row)
row = result.fetchone()
print(row)
row = result.fetchone()
print(row)
row = result.fetchone()
print(row)

2018-07-15 21:08:35,250 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-07-15 21:08:35,252 INFO sqlalchemy.engine.base.Engine ()
(1, None, None)
(2, 'lee', 'Unknown, Lee')
(3, 'xx', 'xxxx, xxx')
None
None


fetchone: 테이블 내 검색 결과를 한줄만 가져옴.

테이블 내용 수 초과하여 실행시 None 리턴.

In [227]:
result = conn.execute(query)
rows = result.fetchall()
for row in rows:
    print(row)

2018-07-15 21:08:42,753 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-07-15 21:08:42,756 INFO sqlalchemy.engine.base.Engine ()
(1, None, None)
(2, 'lee', 'Unknown, Lee')
(3, 'xx', 'xxxx, xxx')


fetchall: 테이블 내 검색 결과 전부 가져옴.

In [228]:
for row in rows:
    print(row[0], row[1], row[2])

1 None None
2 lee Unknown, Lee
3 xx xxxx, xxx


list형태로도 접근 가능.

In [147]:
result = conn.execute(select([users.c.name, users.c.fullname]))

2018-07-10 16:23:00,652 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2018-07-10 16:23:00,653 INFO sqlalchemy.engine.base.Engine ()


In [148]:
for row in result:
    print(row)

('kim', 'Anonymous, Kim')
('lee', 'Unknown, Lee')
('xx', 'xxxx, xxx')


name, fullname만 특정하여 결과 가져옴

___

### 비교연산

In [149]:
from sqlalchemy import and_, or_, not_

print(users.c.id == addresses.c.user_id)

users.id = addresses.user_id


파이썬 문법 상에서 실행하는것이므로 비교연산시 ==을 사용

In [229]:
print(or_(users.c.id == addresses.c.user_id, users.c.id == 1))
print((users.c.id == addresses.c.user_id) | (users.c.id == 1))

users.id = addresses.user_id OR users.id = :id_1
users.id = addresses.user_id OR users.id = :id_1


파이썬 문법에 의한 논리 연산자만으로도 SQL문 작성 가능.(or_ 안쓰고도 같은 결과 만들어냈다)

In [152]:
result = conn.execute(select([users]).where(users.c.id==1))

2018-07-10 16:23:01,320 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.id = ?
2018-07-10 16:23:01,323 INFO sqlalchemy.engine.base.Engine (1,)


In [153]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, Kim')


qmark 스타일로 where절 실행

In [154]:
result = conn.execute(select([users, addresses]).where(users.c.id == addresses.c.user_id))

2018-07-10 16:23:01,569 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2018-07-10 16:23:01,571 INFO sqlalchemy.engine.base.Engine ()


In [155]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, Kim', 1, 1, 'anonymous.kim@test,com')
(2, 'lee', 'Unknown, Lee', 2, 2, 'unknown.lee@test.com')


join 문법을 사용하지 않고 where만으로 가져왔다.

In [156]:
result = conn.execute(select([users]).where(users.c.name.like('k%')))

2018-07-10 16:23:01,825 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.name LIKE ?
2018-07-10 16:23:01,827 INFO sqlalchemy.engine.base.Engine ('k%',)


In [157]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, Kim')


In [158]:
result = conn.execute(select([users]).where(users.c.name.like('%i%')))

2018-07-10 16:23:01,994 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.name LIKE ?
2018-07-10 16:23:01,995 INFO sqlalchemy.engine.base.Engine ('%i%',)


In [159]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, Kim')


LIKE 조건문 실행

In [160]:
result = conn.execute(select([users, addresses]).where(users.c.id == addresses.c.user_id).where(users.c.name.like('k%')))

2018-07-10 16:23:02,169 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id AND users.name LIKE ?
2018-07-10 16:23:02,171 INFO sqlalchemy.engine.base.Engine ('k%',)


In [161]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, Kim', 1, 1, 'anonymous.kim@test,com')


where에 where를 붙인 형태도 가능.

___

### Join

In [162]:
from sqlalchemy import join

print(users.join(addresses))

users JOIN addresses ON users.id = addresses.user_id


foreign key를 명시하였으므로 foreign key를 조건으로 하여 조인을 실행한다.

In [163]:
print(users.join(addresses, users.c.id == addresses.c.user_id))

users JOIN addresses ON users.id = addresses.user_id


ON 조건을 전부 명시한 형태.

In [164]:
result = conn.execute(select([users]).select_from(users.join(addresses)))

for row in result:
    print(row)

2018-07-10 16:23:02,666 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users JOIN addresses ON users.id = addresses.user_id
2018-07-10 16:23:02,667 INFO sqlalchemy.engine.base.Engine ()
(1, 'kim', 'Anonymous, Kim')
(2, 'lee', 'Unknown, Lee')


In [165]:
result = conn.execute(select([users.c.name, addresses.c.email_address]).select_from(users.join(addresses)))

2018-07-10 16:23:02,753 INFO sqlalchemy.engine.base.Engine SELECT users.name, addresses.email_address 
FROM users JOIN addresses ON users.id = addresses.user_id
2018-07-10 16:23:02,754 INFO sqlalchemy.engine.base.Engine ()


In [166]:
for row in result:
    print(row)

('kim', 'anonymous.kim@test,com')
('lee', 'unknown.lee@test.com')


In [230]:
artist = Table('Artist', metadata,
              Column('id', Integer, primary_key=True),
              Column('name', String, nullable=False))

album = Table('Album', metadata,
             Column('id', Integer, primary_key=True),
             Column('title', String, nullable=False),
             Column('artist_id', Integer, ForeignKey('Artist.id')))

genre = Table('Genre', metadata,
             Column('id', Integer, primary_key=True),
             Column('name', String, nullable=False),
             extend_existing=True)

track = Table('Track', metadata,
             Column('id', Integer, primary_key=True),
             Column('title', String, nullable=False),
             Column('length', Integer, nullable=False),
             Column('rating', Integer, nullable=False),
             Column('count', Integer, nullable=False),
             Column('album_id', Integer, ForeignKey('Album.id')),
             Column('genre_id', Integer, ForeignKey('Genre.id')),
             extend_existing=True)

metadata.create_all(engine_review)

2018-07-15 21:19:58,548 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-07-15 21:19:58,550 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:19:58,552 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2018-07-15 21:19:58,553 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:19:58,556 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Artist")
2018-07-15 21:19:58,557 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:19:58,558 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2018-07-15 21:19:58,560 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:19:58,561 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Genre")
2018-07-15 21:19:58,563 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:19:58,564 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Track")
2018-07-15 21:19:58,565 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:19:58,567 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Artist" (
	id INTEGER NOT NULL, 


___

### Drop

In [231]:
users.drop(engine_review)

2018-07-15 21:20:18,917 INFO sqlalchemy.engine.base.Engine 
DROP TABLE users
2018-07-15 21:20:18,919 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:20:18,924 INFO sqlalchemy.engine.base.Engine COMMIT


In [232]:
addresses.drop(engine_review)

2018-07-15 21:20:23,001 INFO sqlalchemy.engine.base.Engine 
DROP TABLE addresses
2018-07-15 21:20:23,003 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:20:23,006 INFO sqlalchemy.engine.base.Engine COMMIT


DROP TABLE 명령 실행

In [233]:
for row in engine_review.table_names():
    print(row)

2018-07-15 21:21:04,955 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-07-15 21:21:04,958 INFO sqlalchemy.engine.base.Engine ()
Album
Artist
Genre
Track


In [234]:
for row in tables:
    print(row)

users
addresses
Artist
Album
Genre
Track


삭제 안됐음: 엔진에서 바인딩은 해제했으나 메타데이터에서는 아직 삭제하지 않았음.

In [235]:
conn.execute(artist.insert(), [
    {'name':'Led Zepplin'},
    {'name':'AC/DC'}
])

conn.execute(album.insert(), [
    {'title':'IV', 'artist_id':1},
    {'title':'Who Made Who', 'artist_id':2}
])

conn.execute(genre.insert(), [
    {'name':'Rock'},
    {'name':'Metal'}
])

conn.execute(track.insert(),[
    {'title':'Black Dog', 'rating':5, 'length':297, 'count':0, 'album_id':1, 'genre_id':1},
    {'title':'Stairway', 'rating':5, 'length':482, 'count':0, 'album_id':1, 'genre_id':1},
    {'title':'About to rock', 'rating':5, 'length':313, 'count':0, 'album_id':2, 'genre_id':2},
    {'title':'Who Made Who', 'rating':5, 'length':297, 'count':0, 'album_id':2, 'genre_id':2}
])

2018-07-15 21:21:43,023 INFO sqlalchemy.engine.base.Engine INSERT INTO "Artist" (name) VALUES (?)
2018-07-15 21:21:43,025 INFO sqlalchemy.engine.base.Engine (('Led Zepplin',), ('AC/DC',))
2018-07-15 21:21:43,028 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-15 21:21:43,032 INFO sqlalchemy.engine.base.Engine INSERT INTO "Album" (title, artist_id) VALUES (?, ?)
2018-07-15 21:21:43,033 INFO sqlalchemy.engine.base.Engine (('IV', 1), ('Who Made Who', 2))
2018-07-15 21:21:43,036 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-15 21:21:43,041 INFO sqlalchemy.engine.base.Engine INSERT INTO "Genre" (name) VALUES (?)
2018-07-15 21:21:43,042 INFO sqlalchemy.engine.base.Engine (('Rock',), ('Metal',))
2018-07-15 21:21:43,045 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-15 21:21:43,050 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" (title, length, rating, count, album_id, genre_id) VALUES (?, ?, ?, ?, ?, ?)
2018-07-15 21:21:43,052 INFO sqlalchemy.engine.base.Engine (('Black Dog', 

In [236]:
result = conn.execute(artist.select())
for row in result:
    print(row)

2018-07-15 21:21:47,522 INFO sqlalchemy.engine.base.Engine SELECT "Artist".id, "Artist".name 
FROM "Artist"
2018-07-15 21:21:47,524 INFO sqlalchemy.engine.base.Engine ()
(1, 'Led Zepplin')
(2, 'AC/DC')


In [237]:
result = conn.execute(track.select())
for row in result:
    print(row)

2018-07-15 21:21:53,927 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track"
2018-07-15 21:21:53,928 INFO sqlalchemy.engine.base.Engine ()
(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 1)
(3, 'About to rock', 313, 5, 0, 2, 2)
(4, 'Who Made Who', 297, 5, 0, 2, 2)


특정 테이블을 한정시킨 후 select로 결과를 불러온 형태

___

### Update

In [238]:
trackResult = conn.execute(select([track])
                          .where(
                              and_(track.c.album_id == 1, track.c.genre_id == 1)
                          ))

for row in trackResult:
    print(row)

2018-07-15 21:21:55,683 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" 
WHERE "Track".album_id = ? AND "Track".genre_id = ?
2018-07-15 21:21:55,684 INFO sqlalchemy.engine.base.Engine (1, 1)
(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 1)


In [239]:
trackResult = conn.execute(select([track])
                          .where(
                              and_(track.c.album_id == 1, 
                                   or_(
                                       track.c.genre_id == 1,
                                       track.c.genre_id == 2,
                                   )
                              )
                          )
                          )

for row in trackResult:
    print(row)

2018-07-15 21:21:58,981 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" 
WHERE "Track".album_id = ? AND ("Track".genre_id = ? OR "Track".genre_id = ?)
2018-07-15 21:21:58,983 INFO sqlalchemy.engine.base.Engine (1, 1, 2)
(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 1)


In [240]:
from sqlalchemy import update

conn.execute(track.update().values(genre_id = 2).where(track.c.id == 2))
conn.execute(track.update().values(genre_id = 1).where(track.c.id == 3))

2018-07-15 21:22:00,905 INFO sqlalchemy.engine.base.Engine UPDATE "Track" SET genre_id=? WHERE "Track".id = ?
2018-07-15 21:22:00,907 INFO sqlalchemy.engine.base.Engine (2, 2)
2018-07-15 21:22:00,910 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-15 21:22:00,913 INFO sqlalchemy.engine.base.Engine UPDATE "Track" SET genre_id=? WHERE "Track".id = ?
2018-07-15 21:22:00,915 INFO sqlalchemy.engine.base.Engine (1, 3)
2018-07-15 21:22:00,918 INFO sqlalchemy.engine.base.Engine COMMIT


update문 실행

In [178]:
trackResult = conn.execute(select([track])
                          .where(
                              and_(track.c.album_id == 1, track.c.genre_id == 1)
                          ))

for row in trackResult:
    print(row)

2018-07-10 16:23:05,737 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" 
WHERE "Track".album_id = ? AND "Track".genre_id = ?
2018-07-10 16:23:05,738 INFO sqlalchemy.engine.base.Engine (1, 1)
(1, 'Black Dog', 297, 5, 0, 1, 1)


In [179]:
conn.execute(track.update().values(genre_id = 1).where(track.c.id == 2))
conn.execute(track.update().values(genre_id = 1).where(track.c.id == 3))

2018-07-10 16:23:05,937 INFO sqlalchemy.engine.base.Engine UPDATE "Track" SET genre_id=? WHERE "Track".id = ?
2018-07-10 16:23:05,938 INFO sqlalchemy.engine.base.Engine (1, 2)
2018-07-10 16:23:05,940 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-10 16:23:05,944 INFO sqlalchemy.engine.base.Engine UPDATE "Track" SET genre_id=? WHERE "Track".id = ?
2018-07-10 16:23:05,945 INFO sqlalchemy.engine.base.Engine (1, 3)
2018-07-10 16:23:05,947 INFO sqlalchemy.engine.base.Engine COMMIT


In [180]:
trackResult = conn.execute(select([track])
                          .where(
                              and_(track.c.album_id == 1, track.c.genre_id == 1)
                          ))

for row in trackResult:
    print(row)

2018-07-10 16:23:06,669 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" 
WHERE "Track".album_id = ? AND "Track".genre_id = ?
2018-07-10 16:23:06,672 INFO sqlalchemy.engine.base.Engine (1, 1)
(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 1)


In [181]:
print(track.join(album))

"Track" JOIN "Album" ON "Album".id = "Track".album_id


In [182]:
result = conn.execute(track.select().select_from(track.join(album)))

for row in result:
    print(row)

2018-07-10 16:23:07,289 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id
2018-07-10 16:23:07,291 INFO sqlalchemy.engine.base.Engine ()
(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 1)
(3, 'About to rock', 313, 5, 0, 2, 1)
(4, 'Who Made Who', 297, 5, 0, 2, 2)


join문 실행

In [242]:
result = conn.execute(select([track.c.title, album.c.title]).select_from(track.join(album)))

for row in result:
    print(row)

2018-07-15 21:23:23,281 INFO sqlalchemy.engine.base.Engine SELECT "Track".title, "Album".title 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id
2018-07-15 21:23:23,285 INFO sqlalchemy.engine.base.Engine ()
('Black Dog', 'IV')
('Stairway', 'IV')
('About to rock', 'Who Made Who')
('Who Made Who', 'Who Made Who')


In [241]:
print(track.join(album))
print(track.join(album).join(genre))
print(track.join(album).join(genre).join(artist))

"Track" JOIN "Album" ON "Album".id = "Track".album_id
"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id
"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id


In [243]:
result = conn.execute(select([track.c.title, album.c.title, genre.c.name, artist.c.name]).select_from(track.join(album).join(genre).join(artist)))

for row in result:
    print(row)

2018-07-15 21:23:31,791 INFO sqlalchemy.engine.base.Engine SELECT "Track".title, "Album".title, "Genre".name, "Artist".name 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id
2018-07-15 21:23:31,795 INFO sqlalchemy.engine.base.Engine ()
('Black Dog', 'IV', 'Rock', 'Led Zepplin')
('Stairway', 'IV', 'Metal', 'Led Zepplin')
('About to rock', 'Who Made Who', 'Rock', 'AC/DC')
('Who Made Who', 'Who Made Who', 'Metal', 'AC/DC')


In [244]:
result = conn.execute(select([track.c.title, album.c.title, genre.c.name, artist.c.name]).select_from(
        track.join(album).join(genre).join(artist)
    ).where(
        and_(
            genre.c.id == 1,
            artist.c.name.like('%pp%'),
        )
    ))

for row in result:
    print(row)

2018-07-15 21:24:13,578 INFO sqlalchemy.engine.base.Engine SELECT "Track".title, "Album".title, "Genre".name, "Artist".name 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id 
WHERE "Genre".id = ? AND "Artist".name LIKE ?
2018-07-15 21:24:13,581 INFO sqlalchemy.engine.base.Engine (1, '%pp%')
('Black Dog', 'IV', 'Rock', 'Led Zepplin')


Multiple join 실행 - 테이블 3개를 동시에 join.

___

## 저장된 것 불러오기

In [245]:
from sqlalchemy import create_engine, MetaData

engine = create_engine('sqlite:///alchemy.db', echo=True)
conn = engine.connect()

2018-07-15 21:24:31,125 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-15 21:24:31,126 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:24:31,127 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-15 21:24:31,128 INFO sqlalchemy.engine.base.Engine ()


In [246]:
metadata = MetaData(bind=engine, reflect=True)

2018-07-15 21:24:35,815 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-07-15 21:24:35,817 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:24:35,820 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2018-07-15 21:24:35,822 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:24:35,825 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Album' AND type = 'table'
2018-07-15 21:24:35,826 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:24:35,831 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("Album")
2018-07-15 21:24:35,832 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:24:35,835 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Album' AND type = 'table'
2018-07-15 21:24:35,837 INFO sqlalchemy.engine.base.Engine ()
2018-07

/Users/keith_lee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  """Entry point for launching an IPython kernel.


sqlite_master(숨겨진 테이블)에서 모든 테이블 정보를 이름순으로 가져왔다.

warning: reflect를 인자로 주지 말고 메소드를 사용하는 것을 권장.

In [247]:
metadata.reflect(bind=engine)

2018-07-15 21:24:59,089 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-07-15 21:24:59,092 INFO sqlalchemy.engine.base.Engine ()


In [248]:
for row in metadata.tables:
    print(row)

Album
Artist
Genre
Track


물리적으로 존재하는 DB(alchemytest.db)에서 4개의 테이블을 메모리로 가져왔다.

In [249]:
album = metadata.tables['Album']

In [250]:
result = conn.execute(album.select())

2018-07-15 21:25:05,939 INFO sqlalchemy.engine.base.Engine SELECT "Album".id, "Album".title, "Album".artist_id 
FROM "Album"
2018-07-15 21:25:05,943 INFO sqlalchemy.engine.base.Engine ()


In [251]:
for row in result:
    print(row)

(1, 'IV', 1)
(2, 'Who Made Who', 2)


___